# Preprocesamiento para funciones API

Antes de comenzar, simplificaremos todo al mayor nivel posible.

Para eso analicemos que vamos a necesitar como resultado final:
<ul> <li> 1-PlayTimeGenre: Debe devolver año con mas horas jugadas para dicho género. -<b> Horas jugadas por genero y anio.</b></li>
<li>2-UserForGenre: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. -<b> Horas jugadas por usuario por genero por anio.</b></li>
<li>3-UsersRecommend: Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales) -<b> Recomendaciones y sentimientos por juego por anio </b></li>
<li>4-UsersWorstDeveloper: Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos) -<b>Recomendaciones segun desarrolladora por anio </b></li>
<li>5-sentiment_analysis: Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor. -<b> Sentimientos por desarrolladora.</b></li></ul>

Pasando en limpio, necesitamos las siguientes tablas madres:
<ul><li>1 y 2: playtime_forever por genre por year por user_id - Llamaremos a este df df_playtime</li>
<li>3, 4 y 5:recommend y sentimientos por desarrolladora por item_name por year - Llamaremos a este df df_feel</li>


In [1]:
import pandas as pd
import Modulo as md

In [2]:
df_items = pd.read_parquet('Datasets/parquet/users_items.parquet')
df_items

,user_id,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,76561197970982479,10,Counter-Strike,6
1,76561197970982479,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,76561197970982479,30,Day of Defeat,7
3,76561197970982479,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...,...
5094100,76561198329548331,76561198329548331,346330,BrainBread 2,0
5094101,76561198329548331,76561198329548331,373330,All Is Dust,0
5094102,76561198329548331,76561198329548331,388490,One Way To Die: Steam Edition,3
5094103,76561198329548331,76561198329548331,521570,You Have 10 Seconds 2,4


In [3]:
df_reviews = pd.read_parquet('Datasets/parquet/user_reviews_sentiments.parquet')
df_reviews

,user_id,item_id,recommend,posted,sentiment_analysis
0,76561197970982479,1250,True,"Posted November 5, 2011.",1
1,76561197970982479,22200,True,"Posted July 15, 2011.",2
2,76561197970982479,43110,True,"Posted April 21, 2011.",1
3,js41637,251610,True,"Posted June 24, 2014.",1
4,js41637,227300,True,"Posted September 8, 2013.",1
...,...,...,...,...,...
58426,76561198312638244,70,True,Posted July 10.,2
58427,76561198312638244,362890,True,Posted July 8.,1
58428,LydiaMorley,273110,True,Posted July 3.,1
58429,LydiaMorley,730,True,Posted July 20.,2


In [4]:
df_games = pd.read_parquet('Datasets/parquet/steam_games.parquet')
df_games

,genres,price,id,developer,app_name,release_year
0,Action,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
1,Casual,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
2,Indie,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
3,Simulation,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
4,Strategy,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
...,...,...,...,...,...,...
71546,Indie,1.99,610660,Laush Dmitriy Sergeevich,Russian Roads,2018
71547,Racing,1.99,610660,Laush Dmitriy Sergeevich,Russian Roads,2018
71548,Simulation,1.99,610660,Laush Dmitriy Sergeevich,Russian Roads,2018
71549,Casual,4.99,658870,"xropi,stev3ns",EXIT 2 - Directions,2017


## df_playtime

Recordemos la definicion: playtime_forever por genre por year por user_id.

Para este caso, solo debemos usar df_items y el release year de df_games.

In [5]:
df_items

,user_id,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,76561197970982479,10,Counter-Strike,6
1,76561197970982479,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,76561197970982479,30,Day of Defeat,7
3,76561197970982479,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,76561197970982479,50,Half-Life: Opposing Force,0
...,...,...,...,...,...
5094100,76561198329548331,76561198329548331,346330,BrainBread 2,0
5094101,76561198329548331,76561198329548331,373330,All Is Dust,0
5094102,76561198329548331,76561198329548331,388490,One Way To Die: Steam Edition,3
5094103,76561198329548331,76561198329548331,521570,You Have 10 Seconds 2,4


In [6]:
df_games

,genres,price,id,developer,app_name,release_year
0,Action,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
1,Casual,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
2,Indie,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
3,Simulation,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
4,Strategy,4.99,761140,Kotoshiro,Lost Summoner Kitty,2018
...,...,...,...,...,...,...
71546,Indie,1.99,610660,Laush Dmitriy Sergeevich,Russian Roads,2018
71547,Racing,1.99,610660,Laush Dmitriy Sergeevich,Russian Roads,2018
71548,Simulation,1.99,610660,Laush Dmitriy Sergeevich,Russian Roads,2018
71549,Casual,4.99,658870,"xropi,stev3ns",EXIT 2 - Directions,2017


In [7]:
df_games.rename(columns={'id':'item_id'},inplace=True)

In [8]:
df_pre_playtime = df_games[['item_id','release_year','genres']].merge(df_items[['item_id','user_id','playtime_forever']], on='item_id')
df_pre_playtime

,item_id,release_year,genres,user_id,playtime_forever
0,282010,1997,Action,UTNerd24,5
1,282010,1997,Action,I_DID_911_JUST_SAYING,0
2,282010,1997,Action,76561197962104795,0
3,282010,1997,Action,r3ap3r78,0
4,282010,1997,Action,saint556,13
...,...,...,...,...,...
9877299,80,2004,Action,76561198273508956,0
9877300,80,2004,Action,76561198282090798,0
9877301,80,2004,Action,943525,0
9877302,80,2004,Action,76561198283312749,9


In [9]:
df_pre_playtime = df_pre_playtime[df_pre_playtime['release_year'] != 0] 
#eliminamos los release year = 0 ya que son los que no tienen anio de release y entonces no nos sirven para las funciones solicitadas

In [10]:
df_playtime = df_pre_playtime.groupby(['genres','release_year','user_id'])['playtime_forever'].sum().reset_index()

In [11]:
df_playtime

,genres,release_year,user_id,playtime_forever
0,Action,1983,2Ta4,18
1,Action,1983,76561197960324641,0
2,Action,1983,76561197966936422,331
3,Action,1983,76561197968887720,1
4,Action,1983,76561197969020980,98
...,...,...,...,...
3429398,Web Publishing,2017,Eosoforcus,58
3429399,Web Publishing,2017,N47H4NI3L,1635
3429400,Web Publishing,2017,dirklah,796
3429401,Web Publishing,2017,kushziller,251


In [12]:
df_playtime.to_parquet('Datasets/parquet/API/playtime.parquet',index=False)

## df_feel

Recordemos la definicion: recommend y sentimientos por desarrolladora por item_name por year

Para ello, solo debemos usar df_reviews y df_games.

In [13]:
df_reviews = df_reviews[['recommend','sentiment_analysis','item_id']]
df_reviews

,recommend,sentiment_analysis,item_id
0,True,1,1250
1,True,2,22200
2,True,1,43110
3,True,1,251610
4,True,1,227300
...,...,...,...
58426,True,2,70
58427,True,1,362890
58428,True,1,273110
58429,True,2,730


In [14]:
df_games = df_games[['app_name','release_year','item_id','developer']]
df_games.rename(columns={'id':'item_id'},inplace=True)
df_games

C:\Users\pedro\AppData\Local\Temp\ipykernel_17060\1087801686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games.rename(columns={'id':'item_id'},inplace=True)


,app_name,release_year,item_id,developer
0,Lost Summoner Kitty,2018,761140,Kotoshiro
1,Lost Summoner Kitty,2018,761140,Kotoshiro
2,Lost Summoner Kitty,2018,761140,Kotoshiro
3,Lost Summoner Kitty,2018,761140,Kotoshiro
4,Lost Summoner Kitty,2018,761140,Kotoshiro
...,...,...,...,...
71546,Russian Roads,2018,610660,Laush Dmitriy Sergeevich
71547,Russian Roads,2018,610660,Laush Dmitriy Sergeevich
71548,Russian Roads,2018,610660,Laush Dmitriy Sergeevich
71549,EXIT 2 - Directions,2017,658870,"xropi,stev3ns"


In [15]:
df_pre_feel = pd.merge(df_reviews,df_games,on='item_id')
df_pre_feel

,recommend,sentiment_analysis,item_id,app_name,release_year,developer
0,True,1,1250,Killing Floor,2009,Tripwire Interactive
1,True,2,1250,Killing Floor,2009,Tripwire Interactive
2,True,1,1250,Killing Floor,2009,Tripwire Interactive
3,True,1,1250,Killing Floor,2009,Tripwire Interactive
4,True,1,1250,Killing Floor,2009,Tripwire Interactive
...,...,...,...,...,...,...
124306,True,0,262850,The Journey Down: Chapter Two,2014,SkyGoblin
124307,True,2,431510,Mystic Destinies: Serendipity of Aeons,2016,Aeon Dream Studios
124308,True,2,431510,Mystic Destinies: Serendipity of Aeons,2016,Aeon Dream Studios
124309,True,2,431510,Mystic Destinies: Serendipity of Aeons,2016,Aeon Dream Studios


In [16]:
df_feel = df_pre_feel.groupby(['developer','release_year','app_name','sentiment_analysis','recommend']).count().reset_index()
df_feel

,developer,release_year,app_name,sentiment_analysis,recommend,item_id
0,07th Expansion,2015,Higurashi When They Cry Hou - Ch.1 Onikakushi,0,True,1
1,07th Expansion,2015,Higurashi When They Cry Hou - Ch.1 Onikakushi,1,True,1
2,07th Expansion,2016,Umineko When They Cry - Question Arcs,1,True,1
3,"10th Art Studio,Adventure Productions",2014,Shadows on the Vatican Act I: Greed,1,False,2
4,"10th Art Studio,Adventure Productions",2014,Shadows on the Vatican Act I: Greed,2,True,2
...,...,...,...,...,...,...
6555,"インレ,Inre",2016,ChuSingura46+1 S,2,True,2
6556,橘子班,2016,Season of 12 Colors,0,True,3
6557,橘子班,2016,Season of 12 Colors,1,True,6
6558,橘子班,2016,Season of 12 Colors,2,True,3


In [17]:
df_feel = df_feel[df_feel['release_year']!=0]
df_feel

,developer,release_year,app_name,sentiment_analysis,recommend,item_id
0,07th Expansion,2015,Higurashi When They Cry Hou - Ch.1 Onikakushi,0,True,1
1,07th Expansion,2015,Higurashi When They Cry Hou - Ch.1 Onikakushi,1,True,1
2,07th Expansion,2016,Umineko When They Cry - Question Arcs,1,True,1
3,"10th Art Studio,Adventure Productions",2014,Shadows on the Vatican Act I: Greed,1,False,2
4,"10th Art Studio,Adventure Productions",2014,Shadows on the Vatican Act I: Greed,2,True,2
...,...,...,...,...,...,...
6555,"インレ,Inre",2016,ChuSingura46+1 S,2,True,2
6556,橘子班,2016,Season of 12 Colors,0,True,3
6557,橘子班,2016,Season of 12 Colors,1,True,6
6558,橘子班,2016,Season of 12 Colors,2,True,3


In [21]:
df_feel.to_parquet('Datasets/parquet/API/feel.parquet',index=False)